
# Graffiti Annual  Numbers

# Make Sure the modules are installed



# Import modules

In [1]:

import numpy as np
import pandas as pd




In [2]:
!pwd

/home/gdillon/JupyterPractice/graffiti_SF311


# produce viz of annual total 2009-2016
because I'm using SQL type queery with count(*), this responds faster

## api call for count of all by year

In [3]:
url = 'https://data.sfgov.org/resource/qer8-n8u9.csv?$select=date_trunc_y(opened)as%20year,%20count(*)&$order=year&$group=year'

In [4]:
df = pd.read_csv(url)
df = df[1:-1]
df

,year,count
1,01/01/2009 12:00:00 AM,32285
2,01/01/2010 12:00:00 AM,26769
3,01/01/2011 12:00:00 AM,25445
4,01/01/2012 12:00:00 AM,26254
5,01/01/2013 12:00:00 AM,35831
6,01/01/2014 12:00:00 AM,46808
7,01/01/2015 12:00:00 AM,57249
8,01/01/2016 12:00:00 AM,72392


In [5]:
df['year']=df['year'].str.extract('(?P<year>\d{4})').astype(str)
df['count']=df['count'].astype(int)
df

/home/gdillon/.local/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


,year,count
1,2009,32285
2,2010,26769
3,2011,25445
4,2012,26254
5,2013,35831
6,2014,46808
7,2015,57249
8,2016,72392


In [6]:
from bokeh.charts import Bar, output_file, output_notebook, show
from bokeh.layouts import row
output_notebook()

Loading BokehJS ...

In [7]:
trend = Bar(df, values = 'count', label = 'year', title = 'Graffiti reported to SF311 by year')
output_notebook()
show(row(trend))

Loading BokehJS ...

In [8]:
df = df[3:-1]
df

,year,count
4,2012,26254
5,2013,35831
6,2014,46808
7,2015,57249


In [9]:
df.index

RangeIndex(start=4, stop=8, step=1)

In [10]:
trend = Bar(df, values = 'count', label = 'year')
show(row(trend))

# Analysis by Source, grab new dataframe by constructing SQL query to API

In [11]:
url = 'https://data.sfgov.org/resource/qer8-n8u9.csv?$select=source,date_trunc_y(opened)as%20year,%20count(*)&$order=year&$group=year,source'

In [12]:
print(url)

https://data.sfgov.org/resource/qer8-n8u9.csv?$select=source,date_trunc_y(opened)as%20year,%20count(*)&$order=year&$group=year,source


In [13]:
df=pd.read_csv(url)
df=df[18:]
df['year']=df['year'].str.extract('(?P<year>\d{4})').astype(str)
df['count']=df['count'].astype(int)
df=df[:-1]
#df.head(3)
df

/home/gdillon/.local/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  app.launch_new_instance()


,Source,year,count
18,Open311,2013,4030
19,Integrated Agency,2013,9666
20,Voice In,2013,9293
21,Web Self Service,2013,11330
22,Twitter,2013,1512
23,NaN,2014,5
24,Open311,2014,19813
25,Twitter,2014,2227
26,Web Self Service,2014,11769
27,Integrated Agency,2014,5493


In [14]:
df2 =pd.pivot_table(
  df,values='count',
  index=['year'], 
  columns=['Source'],
  aggfunc=np.sum) 
df2

Source,Integrated Agency,Open311,Other Department,Twitter,Voice In,Web Self Service
year,,,,,,
2013,9666.0,4030.0,NaN,1512.0,9293.0,11330.0
2014,5493.0,19813.0,NaN,2227.0,7501.0,11769.0
2015,8530.0,38732.0,6.0,202.0,5636.0,4143.0
2016,7992.0,55148.0,NaN,194.0,4390.0,4668.0


In [15]:

df2.drop(df2.columns[[2]], axis=1, inplace=True)
df2

Source,Integrated Agency,Open311,Twitter,Voice In,Web Self Service
year,,,,,
2013,9666.0,4030.0,1512.0,9293.0,11330.0
2014,5493.0,19813.0,2227.0,7501.0,11769.0
2015,8530.0,38732.0,202.0,5636.0,4143.0
2016,7992.0,55148.0,194.0,4390.0,4668.0


In [16]:
df2.index

Index(['2013', '2014', '2015', '2016'], dtype='object', name='year')

example code from stackoverflow http://stackoverflow.com/questions/31520951/plotting-multiple-lines-with-bokeh-and-pandas

In [17]:
import numpy as np
from bokeh.charts import Line, output_file, show

# (dict, OrderedDict, lists, arrays and DataFrames are valid inputs)
#xyvalues = np.array([[2, 3, 7, 5, 26], [12, 33, 47, 15, 126], [22, 43, 10, 25, 26]])
xyvalues = df2
line = Line(xyvalues, title="line", legend="top_left", ylabel='Reports to SF311 by Year')

output_file('line.html')
show(line)

# Analysis by Source, grab new dataframe by constructing SQL query to API


# SQL the Responsible Agency

In [18]:
#url = 'https://data.sfgov.org/resource/qer8-n8u9.csv?$select=responsible_agency,date_trunc_y(opened)as%20year,%20count(*)&$order=year&$group=year,responsible_agency'#
#url = 'https://data.sfgov.org/resource/qer8-n8u9.csv?$select=responsible_agency,date_trunc_y(opened)as%20year,%20count(*)&$order=year&$group=year,responsible_agency'
url = 'https://data.sfgov.org/resource/qer8-n8u9.csv?$select=responsible_agency,category,date_trunc_y(opened)as%20year,%20count(*)&$order=year&$group=year,category,responsible_agency'

In [19]:
print(url)

https://data.sfgov.org/resource/qer8-n8u9.csv?$select=responsible_agency,category,date_trunc_y(opened)as%20year,%20count(*)&$order=year&$group=year,category,responsible_agency


In [20]:
df=pd.read_csv(url)
df=df[18:]
df['year']=df['year'].str.extract('(?P<year>\d{4})').astype(str)
#df['count']=df['count'].astype(int)
pd.to_numeric(df['count'])
df=df[:-1]
df.tail(24)

/home/gdillon/.local/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  app.launch_new_instance()


,Responsible Agency,Category,year,count
570,DPW BSM Queue,Graffiti Private Property,2016,260
571,SFMTA - Muni System Feedback Queue,Graffiti Public Property,2016,233
572,DPT Paint Shop Queue,Graffiti Private Property,2016,24
573,311 Service Request Queue - Hold,Graffiti Private Property,2016,76
574,Clear Channel - Transit Queue,Graffiti Private Property,2016,29
575,PUC - Water - Graffiti Queue,Graffiti Public Property,2016,473
576,DPT SignShop Unknown Queue,Graffiti Public Property,2016,30
577,DPT - Bicycle - G,Graffiti Public Property,2016,1
578,DPT SignShop 05 Queue,Graffiti Public Property,2016,459
579,DPT SignShop 02 Queue,Graffiti Public Property,2016,69


In [21]:
df = df[df.Category == 'Graffiti Public Property']
#df = df[df.year == '2016']
df = df[(df.year == '2015') | (df.year == '2016')  ]
df = df[df['count'] >=500]
#df = df[df['Responsible Agency'].str.contains('DPT')]
df = df[-df['Responsible Agency'].str.contains('311')]
df = df[-df['Responsible Agency'].str.contains('PUC')]
df = df[-df['Responsible Agency'].str.contains('DPW')]

df

,Responsible Agency,Category,year,count
421,DPT SignShop Surveyed - Area 1 Queue,Graffiti Public Property,2015,544
430,Clear Channel - Transit Queue,Graffiti Public Property,2015,2511
432,DPT Meter_Bike Queue,Graffiti Public Property,2015,3092
433,US Postal Service Maintenance Queue,Graffiti Public Property,2015,1624
586,Clear Channel - Transit Queue,Graffiti Public Property,2016,3933
587,DPT Meter_Bike Queue,Graffiti Public Property,2016,3132
588,US Postal Service Maintenance Queue,Graffiti Public Property,2016,5242


In [22]:
df['count'] = df['count'].apply(pd.to_numeric)

In [23]:
df = df[df['count'] >= 800]


In [24]:
df2 =pd.pivot_table(
  df,values='count',
  index=['year'], 
  columns=['Responsible Agency'],
  aggfunc=np.sum) 
df2

Responsible Agency,Clear Channel - Transit Queue,DPT Meter_Bike Queue,US Postal Service Maintenance Queue
year,,,
2015,2511,3092,1624
2016,3933,3132,5242


In [25]:
df.shape

(6, 4)

In [26]:
import numpy as np
from bokeh.charts import Line, output_file, show

# (dict, OrderedDict, lists, arrays and DataFrames are valid inputs)
#xyvalues = np.array([[2, 3, 7, 5, 26], [12, 33, 47, 15, 126], [22, 43, 10, 25, 26]])
xyvalues = df2
line = Line(xyvalues, title="line", legend="top_left", ylabel='Non-DPW Big Agencies', xlabel='Year to Year')

output_file('line.html')
show(line)

INFO:bokeh.core.state:Session output file 'line.html' already exists, will be overwritten.


# Create Graph by Supervisor District

In [53]:
## Top 5 and Bottom 6 Separated

### Construct API Endpoint with SQL like Query.   (similar to above)

In [76]:
#url = 'https://data.sfgov.org/resource/qer8-n8u9.csv?$select=responsible_agency,date_trunc_y(opened)as%20year,%20count(*)&$order=year&$group=year,responsible_agency'#
#url = 'https://data.sfgov.org/resource/qer8-n8u9.csv?$select=responsible_agency,date_trunc_y(opened)as%20year,%20count(*)&$order=year&$group=year,responsible_agency'
url = 'https://data.sfgov.org/resource/qer8-n8u9.csv?$select=supervisor_district,category,date_trunc_y(opened)as%20year,%20count(*)&$order=year&$group=year,category,supervisor_district'

In [77]:
df=pd.read_csv(url)
df=df[18:]
df['year']=df['year'].str.extract('(?P<year>\d{4})').astype(str)
df['count']=df['count'].astype(int)
df=df[:-1]
#df.head(3)
df

/home/gdillon/.local/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  app.launch_new_instance()


,Supervisor District,Category,year,count
18,1.0,Graffiti Public Property,2008,306
19,7.0,Graffiti Private Property,2008,362
20,5.0,Graffiti Public Property,2008,476
21,5.0,Graffiti Private Property,2008,592
22,0.0,Graffiti Private Property,2009,2
23,10.0,Graffiti Public Property,2009,530
24,4.0,Graffiti Public Property,2009,391
25,1.0,Graffiti Public Property,2009,1679
26,8.0,Graffiti Private Property,2009,1149
27,11.0,Graffiti Public Property,2009,2068


In [78]:
df2 =pd.pivot_table(
  df,values='count',
  index=['year'], 
  columns=['Supervisor District'],
  aggfunc=np.sum,
  fill_value=0) 
df2

Supervisor District,-1.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
year,,,,,,,,,,,,,
2008,0,0,306,0,0,0,1068,0,362,0,0,0,0
2009,0,2,3015,1543,4067,861,4714,2676,1354,2315,6658,1369,3711
2010,0,5,3209,1370,1785,1465,5493,1913,926,2290,4821,951,2541
2011,0,6,3769,1320,1845,1241,5397,2483,1000,2592,3560,645,1587
2012,0,13,3223,1632,2957,741,5223,2664,670,2577,4401,845,1308
2013,0,22,3713,1667,3313,769,7023,4612,1038,3990,6310,1325,2049
2014,0,70,4742,2376,4179,1005,9497,6758,1032,6123,7455,1891,1680
2015,0,631,3914,2483,5563,1591,11421,7442,1165,6686,11290,3370,1693
2016,10,740,4644,2345,11836,1345,11913,9701,1297,8755,13981,3938,1850


In [79]:
df2.drop(df2.columns[[0,1]], axis=1, inplace=True)


df2


Supervisor District,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
year,,,,,,,,,,,
2008,306,0,0,0,1068,0,362,0,0,0,0
2009,3015,1543,4067,861,4714,2676,1354,2315,6658,1369,3711
2010,3209,1370,1785,1465,5493,1913,926,2290,4821,951,2541
2011,3769,1320,1845,1241,5397,2483,1000,2592,3560,645,1587
2012,3223,1632,2957,741,5223,2664,670,2577,4401,845,1308
2013,3713,1667,3313,769,7023,4612,1038,3990,6310,1325,2049
2014,4742,2376,4179,1005,9497,6758,1032,6123,7455,1891,1680
2015,3914,2483,5563,1591,11421,7442,1165,6686,11290,3370,1693
2016,4644,2345,11836,1345,11913,9701,1297,8755,13981,3938,1850


In [82]:
df2

Supervisor District,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
year,,,,,,,,,,,
2008,306,0,0,0,1068,0,362,0,0,0,0
2009,3015,1543,4067,861,4714,2676,1354,2315,6658,1369,3711
2010,3209,1370,1785,1465,5493,1913,926,2290,4821,951,2541
2011,3769,1320,1845,1241,5397,2483,1000,2592,3560,645,1587
2012,3223,1632,2957,741,5223,2664,670,2577,4401,845,1308
2013,3713,1667,3313,769,7023,4612,1038,3990,6310,1325,2049
2014,4742,2376,4179,1005,9497,6758,1032,6123,7455,1891,1680
2015,3914,2483,5563,1591,11421,7442,1165,6686,11290,3370,1693
2016,4644,2345,11836,1345,11913,9701,1297,8755,13981,3938,1850


In [30]:
# drop from url those prior to January 1
where = '&$where='
category = 'opened'
start = '>='

#date format 'year-mm-dd'
onorafter = "'2016-01-01'"
url2 = url2 + where + category + start + str(onorafter) 
print(url2)

NameError: name 'url2' is not defined

# Concatenate or Vertically stack the two dataFrames

In [ ]:
df1  = pd.read_csv(url1)
df2 = pd.read_csv(url2)
df = pd.concat([df1,df2])
df.shape


In [ ]:
# frames = [df1,df2]
# df = pd.concat(frames)

# Convert the time series into pandas time version

In [ ]:
df.columns

In [ ]:
type(df.Opened)

In [ ]:
df['Opened'] = pd.to_datetime(df['Opened']).dt.strftime('%Y-%m-%d')

In [ ]:
df['Closed'] = pd.to_datetime(df['Closed']).dt.strftime('%Y-%m-%d')
df['Updated'] = pd.to_datetime(df['Updated']).dt.strftime('%Y-%m-%d')


In [ ]:
df.head(2)

In [ ]:
df.to_string(columns='Supervisor District')

In [ ]:
df.to_string(columns = 'Supervisor District')

In [ ]:
df['Diff'] =(df.Closed-df.Opened).astype('timedelta64[D]')


In [ ]:
type(df['Supervisor District'])

In [ ]:
df.Diff.describe()

In [ ]:
df[df['Neighborhood' ]== "Mission"].head(85)

In [ ]:
df[df['Neighborhood' ]== "Mission"].describe()

In [ ]:
df['Responsible Agency'].value_counts()[:6]

In [ ]:
df[df['Responsible Agency'] == 'US Postal Service Maintenance Queue']['Diff'].describe()

In [ ]:
df[df['Responsible Agency'] == 'Clear Channel - Transit Queue']['Diff'].describe()

In [ ]:
df[df['Responsible Agency'] == 'DPW Ops Queue']['Diff'].describe()

In [ ]:
df[df['Category'] == 'Graffiti Private Property']['Diff'].describe()

In [ ]:
df[df['Category'] == 'Graffiti Public Property']['Diff'].describe()

In [ ]:
df['Source'].value_counts()

In [ ]:
df[df['Source'] == 'Open311']['Diff'].describe()

In [ ]:
df[df['Source'] == 'Voice In']['Diff'].describe()

In [ ]:
df[df['Source'] == 'Integrated Agency']['Diff'].describe()

# Consider some exploratory statistics / reality checks

In [ ]:
source = df.Source.value_counts()
print(type(source))


print(source)

In [ ]:
type(source)

In [ ]:
label_list = source.index.tolist()
print(label_list)

In [ ]:
source = source.tolist()
source


In [ ]:


plt.axis("equal")
plt.pie(
        source,
        labels=label_list,
        autopct="%1.1f%%"
        )
plt.title("Graffiti Reports Submitted in 2016")
plt.show()

In [ ]:
sups = df['Supervisor District'].value_counts().head(11)

In [ ]:
top = 

# Get list of Supervisor's'

In [ ]:
df['Status Notes'].value_counts().head(10)

In [ ]:
df['Status'].value_counts().head()

In [ ]:
df['Responsible Agency'].value_counts().head(8)

In [ ]:
df.Category.value_counts().head()

In [ ]:
df['Request Type'].value_counts().head(10)

In [ ]:
df['Request Details'].value_counts().head(20)

In [ ]:
df['Address'].value_counts().head(20)

In [ ]:
df['Source'].value_counts().head(6)

In [ ]:
df['Media URL'].describe()

In [ ]:
df['Point'].describe()

# Plot this.    Simple First


In [ ]:
#Plot by top responsible agency that is not DPW

# Start Plotting the simple items

# Plot Responsible Agency non DPW

## minimally acceptable is to do pie / dpw vs all other
### DPW share compared to prior years
### Conclude, DPW doing a good job, how about the other departments
### Bar Chart the top 5-6
### Which of the top 5-6 are doing better.
### can't give the most natural statistic, because 